In [1]:
library(tidyverse, quietly = TRUE)
library(psych, quietly = TRUE)
library(Hmisc, quietly = TRUE)
library(car)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘psych’


The following objects are masked from ‘package:ggplot2’:

    %+%, alpha



Attaching package: ‘Hmisc’


The following object is masked from ‘package:psych’:

    describe


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:psych’:

In [2]:
data <- read.csv("../data/combined_data.csv")
head(data, 1)

,NAME,GEOID,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms
,<chr>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,"Census Tract 9601, Asotin County, Washington",53003960100,1569,145,1337,73942,12,58,110.3274,41.7821,51.70445


In [3]:
data <- data %>% dplyr::select(-NAME, -GEOID)
scaled_data <- scale(data)
head(data)

,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms
,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1569,145,1337,73942,12,58.00000,110.3274,41.78210,51.70445
2,1780,421,1352,72988,13,60.07692,145.7848,22.88731,24.62630
3,1156,6,396,38077,11,60.00000,173.4079,27.06202,29.98058
4,805,29,647,38777,8,62.50000,127.6005,23.53829,22.16379
5,1198,26,635,42639,8,62.50000,192.4299,21.68565,21.66087
6,1392,115,909,53947,9,58.88889,151.0552,23.52123,26.18557


In [4]:
KMO(data)

Kaiser-Meyer-Olkin factor adequacy
Call: KMO(r = data)
Overall MSA =  0.63
MSA for each item = 
      with_internet      work_from_home sixtyfive_and_older       median_income 
               0.57                0.74                0.55                0.65 
   number_providers    mean_lowest_cost         mean_d_mbps         mean_u_mbps 
               0.71                0.69                0.65                0.60 
        mean_lat_ms 
               0.67 

In [5]:
pca_data <- prcomp(scaled_data, scale = FALSE)

# disable scientific notation
options(scipen = 999)
eigenvectors <- -pca_data$rotation
eigenvalues <- pca_data$sdev^2

index_col <- as.data.frame(-pca_data$x) %>%
    dplyr::select(PC1) %>%
    rename(index = PC1)

index_data <- bind_cols(data, index_col)
head(index_data)

,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms,index
,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1569,145,1337,73942,12,58.00000,110.3274,41.78210,51.70445,-1.4973910
2,1780,421,1352,72988,13,60.07692,145.7848,22.88731,24.62630,-0.4445651
3,1156,6,396,38077,11,60.00000,173.4079,27.06202,29.98058,-0.9229616
4,805,29,647,38777,8,62.50000,127.6005,23.53829,22.16379,-0.4642621
5,1198,26,635,42639,8,62.50000,192.4299,21.68565,21.66087,0.1856894
6,1392,115,909,53947,9,58.88889,151.0552,23.52123,26.18557,-0.3719643


In [6]:
as.list(rcorr(as.matrix(index_data), type = "pearson")$P[, ncol(index_data)])

$with_internet
[1] 0.0000000000000184297

$work_from_home
[1] 0.04883481

$sixtyfive_and_older
[1] 0.1971623

$median_income
[1] 0.7867191

$number_providers
[1] 0

$mean_lowest_cost
[1] 0.00000002783109

$mean_d_mbps
[1] 0

$mean_u_mbps
[1] 0.00000000000009725554

$mean_lat_ms
[1] 0

$index
[1] NA

In [7]:
as.list(rcorr(as.matrix(index_data), type = "pearson"))$P

,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms,index
with_internet,NA,0.0000000002997904,0.00000000000000,0.03006990,0.082203264746719906,0.06558715015735,0.000001719724394889,0.00690289845387948020,0.0012541075067722,0.00000000000001842970
work_from_home,0.0000000002997904147,NA,0.00000004534735,0.62244972,0.692796119793766207,0.93350819129855,0.672635578892683217,0.43009326946824133486,0.9085912323262422,0.04883481104472586765
sixtyfive_and_older,0.0000000000000000000,0.0000000453473543,NA,0.04477147,0.476702946467304045,0.49718826894028,0.569411922064795295,0.02203015631983396005,0.1693793288053456,0.19716228248808631562
median_income,0.0300698982044131924,0.6224497181702566,0.04477147246482,NA,0.767570871280282674,0.46966892178772,0.521089097196837159,0.80723851787517308942,0.4313383103711312,0.78671911871383670700
number_providers,0.0822032647467199062,0.6927961197937662,0.47670294646730,0.76757087,NA,0.00000526142273,0.000000000006029399,0.82692993872099873620,0.0000000002296336,0.00000000000000000000
mean_lowest_cost,0.0655871501573528359,0.9335081912985506,0.49718826894028,0.46966892,0.000005261422731273,NA,0.007630242115639785,0.25618495750332925809,0.0141555670114957,0.00000002783108787874
mean_d_mbps,0.0000017197243948885,0.6726355788926832,0.56941192206480,0.52108910,0.000000000006029399,0.00763024211564,NA,0.00000001366549318149,0.0000000000000000,0.00000000000000000000
mean_u_mbps,0.0069028984538794802,0.4300932694682413,0.02203015631983,0.80723852,0.826929938720998736,0.25618495750333,0.000000013665493181,NA,0.0000000029104892,0.00000000000009725554
mean_lat_ms,0.0012541075067722396,0.9085912323262422,0.16937932880535,0.43133831,0.000000000229633645,0.01415556701150,0.000000000000000000,0.00000000291048918299,NA,0.00000000000000000000
index,0.0000000000000184297,0.0488348110447259,0.19716228248809,0.78671912,0.000000000000000000,0.00000002783109,0.000000000000000000,0.00000000000009725554,0.0000000000000000,NA


In [22]:
data1.2 <- data %>% dplyr::select(-median_income, -sixtyfive_and_older)
scaled_data1.2 <- scale(data1.2)

pca_data1.2 <- prcomp(scaled_data1.2, scale = FALSE)
eigenvectors1.2 <- -pca_data1.2$rotation
eigenvalues1.2 <- pca_data1.2$sdev^2

index_col1.2 <- as.data.frame(-pca_data1.2$x) %>%
    dplyr::select(PC1) %>%
    rename(index = PC1)

index_data1.2 <- bind_cols(data1.2, index_col1.2)

In [23]:
as.list(rcorr(as.matrix(index_data1.2), type = "pearson")$P[, ncol(index_data1.2)])

$with_internet
[1] 0.00000000000190048

$work_from_home
[1] 0.12573

$number_providers
[1] 0

$mean_lowest_cost
[1] 0.00000003139127

$mean_d_mbps
[1] 0

$mean_u_mbps
[1] 0.00000000000001976197

$mean_lat_ms
[1] 0

$index
[1] NA

In [24]:
data1.3 <- data %>% dplyr::select(-median_income, -sixtyfive_and_older, -work_from_home)
scaled_data1.3 <- scale(data1.3)

pca_data1.3 <- prcomp(scaled_data1.3, scale = FALSE)
eigenvectors1.3 <- -pca_data1.3$rotation
eigenvalues1.3 <- pca_data1.3$sdev^2

index_col1.3 <- as.data.frame(-pca_data1.3$x) %>%
    dplyr::select(PC1) %>%
    rename(index = PC1)

index_data1.3 <- bind_cols(data1.3, index_col1.3)

In [25]:
as.list(rcorr(as.matrix(index_data1.3), type = "pearson")$P[, ncol(index_data1.3)])

$with_internet
[1] 0.00000000003583245

$number_providers
[1] 0

$mean_lowest_cost
[1] 0.00000002521815

$mean_d_mbps
[1] 0

$mean_u_mbps
[1] 0.000000000000009325873

$mean_lat_ms
[1] 0

$index
[1] NA

In [26]:
library(leaps)
subset <- regsubsets(index ~ ., index_data1.3)
summary(subset)

Subset selection object
Call: regsubsets.formula(index ~ ., index_data1.3)
6 Variables  (and intercept)
                 Forced in Forced out
with_internet        FALSE      FALSE
number_providers     FALSE      FALSE
mean_lowest_cost     FALSE      FALSE
mean_d_mbps          FALSE      FALSE
mean_u_mbps          FALSE      FALSE
mean_lat_ms          FALSE      FALSE
1 subsets of each size up to 6
Selection Algorithm: exhaustive
         with_internet number_providers mean_lowest_cost mean_d_mbps
1  ( 1 ) " "           " "              " "              "*"        
2  ( 1 ) " "           " "              " "              "*"        
3  ( 1 ) " "           "*"              " "              "*"        
4  ( 1 ) "*"           "*"              " "              " "        
5  ( 1 ) "*"           "*"              " "              "*"        
6  ( 1 ) "*"           "*"              "*"              "*"        
         mean_u_mbps mean_lat_ms
1  ( 1 ) " "         " "        
2  ( 1 ) " "      

In [27]:
forward <- regsubsets(index ~ ., index_data1.3, method = "forward")

results <- summary(forward)

results

Subset selection object
Call: regsubsets.formula(index ~ ., index_data1.3, method = "forward")
6 Variables  (and intercept)
                 Forced in Forced out
with_internet        FALSE      FALSE
number_providers     FALSE      FALSE
mean_lowest_cost     FALSE      FALSE
mean_d_mbps          FALSE      FALSE
mean_u_mbps          FALSE      FALSE
mean_lat_ms          FALSE      FALSE
1 subsets of each size up to 6
Selection Algorithm: forward
         with_internet number_providers mean_lowest_cost mean_d_mbps
1  ( 1 ) " "           " "              " "              "*"        
2  ( 1 ) " "           " "              " "              "*"        
3  ( 1 ) " "           " "              "*"              "*"        
4  ( 1 ) "*"           " "              "*"              "*"        
5  ( 1 ) "*"           "*"              "*"              "*"        
6  ( 1 ) "*"           "*"              "*"              "*"        
         mean_u_mbps mean_lat_ms
1  ( 1 ) " "         " "        
2

In [28]:
which.max(results$adjr2)
which.min(results$bic)
which.max(results$rsq)
which.min(results$cp)

[1] 6

[1] 6

[1] 6

[1] 6

In [29]:
coefficients <- coef(forward, 6)
as.list(coefficients[which(coefficients < .05)])

$`(Intercept)`
[1] -4.220056

$with_internet
[1] 0.0005454744

$number_providers
[1] -0.1933058

$mean_d_mbps
[1] 0.006579833

$mean_u_mbps
[1] 0.01236761

$mean_lat_ms
[1] -0.02661983

In [30]:
as.list(coefficients)

$`(Intercept)`
[1] -4.220056

$with_internet
[1] 0.0005454744

$number_providers
[1] -0.1933058

$mean_lowest_cost
[1] 0.07113788

$mean_d_mbps
[1] 0.006579833

$mean_u_mbps
[1] 0.01236761

$mean_lat_ms
[1] -0.02661983

In [31]:
model <- lm(
    index_data1.3$index ~ .,
    data = index_data1.3
)

In [32]:
summary(model)


Call:
lm(formula = index_data1.3$index ~ ., data = index_data1.3)

Residuals:
                   Min                     1Q                 Median 
-0.0000000000000279935 -0.0000000000000000260  0.0000000000000001881 
                    3Q                    Max 
 0.0000000000000004533  0.0000000000000037440 

Coefficients:
                                  Estimate                Std. Error
(Intercept)      -4.2200557226425630119593  0.0000000000000036893039
with_internet     0.0005454743939700425724  0.0000000000000000003572
number_providers -0.1933058388953982353886  0.0000000000000001070922
mean_lowest_cost  0.0711378778822102514345  0.0000000000000000538157
mean_d_mbps       0.0065798325490988678055  0.0000000000000000039673
mean_u_mbps       0.0123676143398365606663  0.0000000000000000075302
mean_lat_ms      -0.0266198346527094509284  0.0000000000000000158545
                           t value            Pr(>|t|)    
(Intercept)      -1143862331586891 <0.0000000000000002 ***
wi

In [33]:
as.list(vif(model))

$with_internet
[1] 1.163424

$number_providers
[1] 1.515062

$mean_lowest_cost
[1] 1.138262

$mean_d_mbps
[1] 3.752037

$mean_u_mbps
[1] 1.320705

$mean_lat_ms
[1] 3.440161